In [7]:
import pandas as pd
import random
import pickle
import json

<h2>Convert rec files for Ranksys</h2>

In [41]:
recs_folder = "/home/diego/chat-rerank/experiments/ml-100k/recs/reranked"
# recs_name = "pzt-fold_0-serendipity-chatgpt"
recs_name = "pzt-fold_0-chatgpt_p1"
top_n = 10

# read JSON file
f = open(f"{recs_folder}/{recs_name}.json")
data = json.load(f)
f.close()

In [42]:
train_folder = f"/home/diego/chat-rerank/experiments/ml-100k/fold_0/train_data.csv"
training_data = pd.read_csv(f'{train_folder}', names=["userid", "itemid", "rating"], sep="\t")

In [44]:
recs = {}
for entry in data:
    recs[entry['userid']] = entry['reranked_recs']
    user_train = list(training_data[training_data["userid"] == entry['userid']]["itemid"].values)
    for item in entry['reranked_recs']:
        if item in user_train:
            # print(entry['userid'])
            print(item)
            print(entry['recs'])
# recs

268
[303, 315, 347, 270, 300, 242, 306, 344, 289, 288, 313, 690, 332, 316, 307, 750, 304, 292, 887, 902, 56, 898, 900, 294, 324, 334, 475, 322, 882, 331, 1022, 354, 197, 271, 875, 995, 895, 937, 538, 752, 343, 879, 246, 754, 751, 299, 100, 886, 906, 350]
268
[272, 286, 435, 346, 333, 344, 483, 194, 271, 304, 332, 879, 326, 523, 750, 260, 174, 898, 310, 301, 288, 484, 651, 191, 182, 751, 137, 23, 246, 83, 357, 475, 175, 661, 186, 324, 689, 292, 197, 508, 334, 283, 880, 487, 902, 11, 8, 124, 514, 275]
268
[300, 286, 307, 475, 742, 316, 7, 748, 751, 628, 302, 271, 321, 285, 750, 591, 117, 269, 1, 689, 327, 323, 343, 310, 13, 319, 14, 886, 895, 410, 678, 222, 273, 147, 282, 151, 93, 763, 127, 332, 347, 50, 342, 655, 357, 1067, 924, 270, 246, 298]
486
[211, 443, 282, 172, 185, 186, 480, 520, 742, 604, 489, 419, 402, 528, 126, 269, 699, 315, 274, 280, 496, 164, 517, 655, 478, 23, 739, 474, 202, 199, 188, 135, 179, 605, 481, 607, 99, 223, 136, 127, 492, 428, 845, 71, 356, 291, 150, 125, 498, 

In [28]:
# print(recs[621])
# print(268 in list(training_data[training_data["userid"] == 621]["itemid"].values))

In [32]:
none_users = 0
users_with_none = 0
total_users = len(recs)
with open(f"{recs_folder}/{recs_name}.csv", "w") as f:
    for user in recs:

        if recs[user] is None:
            none_users += 1
            continue
        score = float(top_n)
        usr_str = ""
        for r in recs[user]:
            if r is None:
                users_with_none += 1
                usr_str = None
                break
            usr_str += f"{user}\t{r}\t{score}\n"
            score -= 1.0
        if usr_str is not None:
            f.write(usr_str)

print(f"Total users: {total_users}")
print(f"None users: {none_users}")
print(f"Users with some None: {users_with_none}")

Total users: 943
None users: 15
Users with some None: 2


<h2>Results analysis</h2>

In [37]:
results_path = "/home/diego/chat-rerank/experiments/ml-100k/results/ranksys_eval"
baseline_recs = pd.read_csv(f"{results_path}/pzt-fold_0")
gpt_recs = pd.read_csv(f"{results_path}/pzt-fold_0-chatgpt.csv")
mmr_recs = pd.read_csv(f"{results_path}/MMR-pzt-fold_0.csv")
xquad_recs = pd.read_csv(f"{results_path}/xQuAD-pzt-fold_0.csv")
rxquad_recs = pd.read_csv(f"{results_path}/RxQuAD-pzt-fold_0.csv")
gpt_seren_recs = pd.read_csv(f"{results_path}/pzt-fold_0-serendipity-chatgpt.csv")

In [38]:
d = {"reranker": ["baseline (MF)", "chat-gpt", "MMR", "xQuAD", "RxQuAD", "chat-gpt-S"]}
print("ChatGPT better than baseline in the following metrics:")
for col in gpt_recs.columns:
    d[col] = [baseline_recs[col][0], gpt_recs[col][0], mmr_recs[col][0], xquad_recs[col][0], rxquad_recs[col][0], gpt_seren_recs[col][0]]
    if gpt_recs[col][0] > baseline_recs[col][0]:
        print(col)
results = pd.DataFrame(data=d)

ChatGPT better than baseline in the following metrics:
s_recall_norel
eild_norel
binomial_norel
cbs_norel
cbs_rel
epc_norel
efd_norel
epd_norel
epd_rel


In [39]:
results

,reranker,prec,recall,ndcg,s_recall_norel,s_recall_rel,a_ndcg,err_ia,eild_norel,eild_rel,binomial_norel,binomial_rel,cbs_norel,cbs_rel,epc_norel,epc_rel,efd_norel,efd_rel,epd_norel,epd_rel
0,baseline (MF),0.227147,0.267024,0.290306,0.501926,0.206117,0.322673,0.155372,0.779424,0.158282,0.396710,0.311174,0.150426,0.022305,0.795201,0.175301,8.874204,1.976686,0.787220,0.181073
1,chat-gpt,0.163203,0.165964,0.207276,0.582519,0.171792,0.250379,0.123352,0.832728,0.112242,0.437842,0.282118,0.196504,0.022438,0.800794,0.125148,8.956339,1.415414,0.824287,0.824287
2,MMR,0.202863,0.237428,0.265213,0.647821,0.208015,0.308786,0.147390,0.908503,0.152934,0.630131,0.342383,NaN,NaN,0.802352,0.156628,8.963094,1.767976,0.845099,0.845099
3,xQuAD,0.219512,0.252236,0.271010,0.599263,0.232293,0.358666,0.173823,0.801650,0.153373,0.349369,0.306487,NaN,NaN,0.791737,0.167227,8.860434,1.895577,0.803579,0.803579
4,RxQuAD,0.224178,0.258343,0.277610,0.554948,0.228107,0.355004,0.180848,0.798137,0.159741,0.359947,0.318495,NaN,NaN,0.791228,0.171029,8.848852,1.937397,0.793750,0.793750
5,chat-gpt-S,0.200848,0.231910,0.253708,0.497293,0.183848,0.285446,0.135465,0.782174,0.136243,0.382823,0.291398,0.145918,0.019126,0.787908,0.155917,8.788886,1.754334,0.790241,0.790241


In [66]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/itemid_to_name.pkl"
with open(f"{filename}", 'rb') as fp:
    itemid_to_name = pickle.load(fp)

In [67]:
itemid_to_name[274]

'Sabrina'

In [68]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/itemname_to_id.pkl"
with open(f"{filename}", 'rb') as fp:
    itemname_to_id = pickle.load(fp)

In [69]:
itemname_to_id["Sabrina"]

486

In [56]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/df_items.csv"
df_items = pd.read_csv(filename)
df_items.head(1)

,movietitle,name,genres
0,Toy Story (1995),Toy Story,"['Animation', 'Childrens', 'Comedy']"


In [58]:
df_items.duplicated().value_counts()

False    1664
True       18
dtype: int64